In [ ]:
# performance results were obtained using cluster
# so here is only the analysis
import os
import matplotlib.pyplot as plt
from collections import defaultdict
from collections import Counter
import statistics

RESULTS_DIR = "../perf_results"
JOBS_DIR = "../perf_jobs"
files = os.listdir(RESULTS_DIR)
# to match jobs and results
print(len(files))

In [ ]:
# I was veeery sleepy when been writing this, so sorry
strlen_to_times = defaultdict(list)
strlen_to_answers = defaultdict(list)

all_strlens = []
all_answers = []
all_groups = []
all_times = []

for fle in files:
    result = os.path.join(RESULTS_DIR, fle)
    job = os.path.join(JOBS_DIR, fle)

    str_lens = []
    answers = []
    times = []

    f = open(job, "r")
    for line in f:
        line_data = line.replace(";", "").split()
        z_in = "-z" in line_data
        f_in = "-f" in line_data
        descr = line_data[3].split("/")[2]
        str_len = int(descr.split("_")[1])
        str_lens.append(str_len)
        all_strlens.append(str_len)
        if z_in and f_in:
            all_groups.append("zf")
        elif z_in:
            all_groups.append("z")
        elif f_in:
            all_groups.append("f")
        else:
            all_groups.append("0")
    f.close()
    f = open(result, "r")
    res_content = f.read()
    f.close()
    res_content_split = [x for x in res_content.split("The answer is:\n") if x != ""]
    for elem in res_content_split:
        elem_split = elem.split("\n")
        answer = True if elem_split[0] == "True" else False
        all_answers.append(answer)
        answers.append(answer)
        real_time = elem_split[2].split("\t")[1]
        m_split = real_time.split("m")
        minutes = int(m_split[0])
        seconds = float(m_split[1][:-1])
        tottime = minutes * 60 + seconds
        times.append(tottime)
        all_times.append(tottime)
    for unit in zip(str_lens, answers, times):
        strlen_to_answers[unit[0]].append(unit[1])
        strlen_to_times[unit[0]].append(unit[2])
# TODO: also consider -f and -z flags

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]

answers_all = flatten(strlen_to_answers.values())
print(Counter(answers_all))
# the worst case

In [ ]:
str_lens = sorted(strlen_to_times.keys())
strlens_false_times = defaultdict(list)
strlens_true_times = defaultdict(list)
means, medians, mins, maxs = [], [], [], []

for str_len in str_lens:
    times = strlen_to_times[str_len]
    answers = strlen_to_answers[str_len]
    mins.append(min(times))
    maxs.append(max(times))
    means.append(statistics.mean(times))
    medians.append(statistics.median(times))
    
    for elem in zip(answers, times):
        if elem[0] is True:
            strlens_true_times[str_len].append(elem[1])
        else:
            strlens_false_times[str_len].append(elem[1])

fig = plt.figure(figsize=(10, 6))
plt.plot(str_lens, means, color="orange", label="Mean val")
plt.plot(str_lens, medians, color="green", label="Median val")
plt.plot(str_lens, maxs, color="red", label="Max val")
plt.title("Overall performance")
plt.xlabel("string length")
plt.ylabel("Time, seconds")
plt.legend()
plt.grid()
plt.show()
# Trues and falses in the same pile

In [ ]:
# let's separate trues and falses
means_f, medians_f, mins_f, maxs_f = [], [], [], []
means_t, medians_t, mins_t, maxs_t = [], [], [], []

for str_len in str_lens:
    true_times = strlens_true_times[str_len]
    false_times = strlens_false_times[str_len]
    
    mins_t.append(min(true_times))
    maxs_t.append(max(true_times))
    means_t.append(statistics.mean(true_times))
    medians_t.append(statistics.median(true_times))

    mins_f.append(min(false_times))
    maxs_f.append(max(false_times))
    means_f.append(statistics.mean(false_times))
    medians_f.append(statistics.median(false_times))

    
fig = plt.figure(figsize=(10, 13))
ax_1 = fig.add_subplot(2, 1, 1)
ax_1.plot(str_lens, means_t, color="orange", label="Mean val")
ax_1.plot(str_lens, medians_t, color="green", label="Median val")
ax_1.plot(str_lens, maxs_t, color="red", label="Max val")
ax_1.set_xlabel("String length")
ax_1.set_ylabel("Time, seconds")
ax_1.set_title("Summary for TRUE answers (best case)")
ax_1.legend()
ax_1.grid()

ax_2 = fig.add_subplot(2, 1, 2)
ax_2.plot(str_lens, means_f, color="orange", label="Mean val")
ax_2.plot(str_lens, medians_f, color="green", label="Median val")
ax_2.plot(str_lens, maxs_f, color="red", label="Max val")
ax_2.plot(str_lens, mins_f, color="grey", label="Min val")
ax_2.set_xlabel("String length")
ax_2.set_ylabel("Time, seconds")
ax_2.set_title("Summary for FALSE answers (worst case)")
ax_2.legend()
ax_2.grid()

In [ ]:
# trues look boring, let's look at falses
false_only = list(filter(lambda x: x[1] is False, zip(all_strlens, all_answers, all_groups, all_times)))
false_only_0 = list(filter(lambda x: x[2] == "0", false_only))
false_only_f = list(filter(lambda x: x[2] == "f", false_only))
false_only_z = list(filter(lambda x: x[2] == "z", false_only))
false_only_zf = list(filter(lambda x: x[2] == "zf", false_only))

print(f"Falses overall: {len(false_only)}")
print(f"Group 0: {len(false_only_0)}")
print(f"Group f: {len(false_only_f)}")
print(f"Group z: {len(false_only_z)}")
print(f"Group zf: {len(false_only_zf)}")
# looks suspicious... -_-

In [ ]:
# strlen, answer, group, time
fig = plt.figure(figsize=(15, 15))
plt.scatter([x[0] for x in false_only_0],
            [x[3] for x in false_only_0],
            label="Group 0", color="grey", alpha=0.5)

plt.scatter([x[0] for x in false_only_f],
            [x[3] for x in false_only_f],
            label="Group F", color="blue", alpha=0.5)

plt.scatter([x[0] for x in false_only_z],
            [x[3] for x in false_only_z],
            label="Group Z", color="green", alpha=0.5)

plt.scatter([x[0] for x in false_only_zf],
            [x[3] for x in false_only_zf],
            label="Group ZF", color="red", alpha=0.5)

plt.xlabel("String length")
plt.ylabel("Time, seconds")
plt.legend()
plt.grid()